# Acrobot Swing Up

### Import Libraries

In [1]:
from copy import copy
import numpy as np
from pydrake.all import (
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
    LeafSystem,
    wrap_to
)
from pydrake.examples import AcrobotGeometry, AcrobotInput, AcrobotPlant, AcrobotState, AcrobotParams

from underactuated import running_as_notebook
from underactuated.meshcat_utils import MeshcatSliders

### Start Meshcat

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://711f3004-0a9b-43a3-abc8-ecd3b79bb722.deepnoteproject.com/7000/


### Balancing LQR Controller

In [3]:
def UprightState():
    state = AcrobotState()
    state.set_theta1(np.pi)
    state.set_theta2(0.0)
    state.set_theta1dot(0.0)
    state.set_theta2dot(0.0)
    return state

In [4]:
def BalancingLQR(acrobot):
    # Design an LQR controller for stabilizing the Acrobot around the upright.
    # Returns a (static) AffineSystem that implements the controller (in
    # the original AcrobotState coordinates).

    context = acrobot.CreateDefaultContext()

    input = AcrobotInput()
    input.set_tau(0.0)
    acrobot.get_input_port(0).FixValue(context, input)

    context.get_mutable_continuous_state_vector().SetFromVector(
        UprightState().CopyToVector()
    )

    Q = np.diag((10.0, 10.0, 1.0, 1.0))
    R = [1]

    # Linearize the dynamics
    linearized_acrobot = Linearize(acrobot, context)
    A = linearized_acrobot.A()
    B = linearized_acrobot.B()

    (K, S) = LinearQuadraticRegulator(
        A, B, Q, R
    )

    return (K, S) 

In [5]:
class BalanceController(LeafSystem):
    def __init__(self, acrobot):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("state", 4)
        self.DeclareVectorOutputPort("control", 1, self.DoCalcOutput)
        (self.K, self.S) = BalancingLQR(acrobot)

    def DoCalcOutput(self, context, output):
        acrobot_state = self.get_input_port(0).Eval(context)
        xbar = copy(acrobot_state)
        xbar[0] = wrap_to(xbar[0], 0, 2.0 * np.pi) - np.pi

        output.SetFromVector(-self.K.dot(xbar))

### Demo of Balancing Controller

In [43]:
builder = DiagramBuilder()
acrobot = builder.AddSystem(AcrobotPlant())

wrapangles = WrapToSystem(4)
wrapangles.set_interval(0, 0, 2.0 * np.pi)
wrapangles.set_interval(1, -np.pi, np.pi)
wrapto = builder.AddSystem(wrapangles)

builder.Connect(acrobot.get_output_port(0), wrapto.get_input_port(0))
controller = builder.AddSystem(BalanceController(acrobot))
builder.Connect(wrapto.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), acrobot.get_input_port(0))

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
AcrobotGeometry.AddToBuilder(builder, acrobot.get_output_port(0), scene_graph)
meshcat.Delete()
meshcat.Set2dRenderMode(xmin=-4, xmax=4, ymin=-4, ymax=4)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

# Simulate
simulator.set_target_realtime_rate(1.0 if running_as_notebook else 0.0)
duration = 4.0 if running_as_notebook else 0.1

context.SetTime(0.0)
context.SetContinuousState(
    UprightState().CopyToVector()
    + [-0.2, 0.2, 0, 0]
)

meshcat.StartRecording()
simulator.AdvanceTo(duration)
meshcat.StopRecording()
meshcat.PublishRecording()

### Energy Shaping Controller

In [70]:
class EnergyShapingController(LeafSystem):
    def __init__(self, acrobot):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("state", 4)
        self.DeclareVectorOutputPort("control", 1, self.DoCalcOutput)
        self.acrobot = acrobot
        self.acrobot_context = acrobot.CreateDefaultContext()
        self.SetAcrobotParams(AcrobotParams())

    def SetAcrobotParams(self, params):
        self.acrobot_context.get_mutable_numeric_parameter(0).SetFromVector(
            params.CopyToVector()
        )
        self.acrobot_context.SetContinuousState([np.pi, 0, 0, 0])
        self.desired_energy = self.acrobot.EvalPotentialEnergy(self.acrobot_context)

    def DoCalcOutput(self, context, output):
        acrobot_state = self.get_input_port(0).Eval(context)
        self.acrobot_context.SetContinuousState(acrobot_state)
        params = self.acrobot_context.get_numeric_parameter(0)

        theta1 = acrobot_state[0]
        theta1dot = acrobot_state[2]
        theta2 = acrobot_state[1]
        theta2dot = acrobot_state[3]

        total_energy = self.acrobot.EvalPotentialEnergy(
            self.acrobot_context
        ) + self.acrobot.EvalKineticEnergy(self.acrobot_context)

        error_energy = total_energy - self.desired_energy
        u_bar = theta1dot * error_energy

        k1 = 0.1
        k2 = 0.1
        k3 = 2.0

        output.SetAtIndex(
            0,
            - k1*theta2 - k2*theta2dot + k3*u_bar
        )

### Swing Up (Energy Shaping) and Balance (LQR) Controller

In [64]:
class SwingUpAndBalanceController(LeafSystem):
    def __init__(self, acrobot):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("state", 4)
        self.DeclareVectorOutputPort("control", 1, self.DoCalcOutput)
        (self.K, self.S) = BalancingLQR(acrobot)
        self.energy_shaping = EnergyShapingController(acrobot)
        self.energy_shaping_context = self.energy_shaping.CreateDefaultContext()

    def DoCalcOutput(self, context, output):
        acrobot_state = self.get_input_port(0).Eval(context)
        xbar = copy(acrobot_state)
        xbar[0] = wrap_to(xbar[0], 0, 2.0 * np.pi) - np.pi

        # If x'Sx <= 300, then use the LQR controller
        if xbar.dot(self.S.dot(xbar)) < 300.0:
            output.SetFromVector(-self.K.dot(xbar))
            print("Using LQR controller")
        else:
            self.energy_shaping.get_input_port(0).FixValue(
                self.energy_shaping_context, acrobot_state
            )
            output.SetFromVector(
                self.energy_shaping.get_output_port(0).Eval(self.energy_shaping_context)
            )

### Demo of Swing Up and Balance Controller

In [75]:
builder = DiagramBuilder()

acrobot = builder.AddSystem(AcrobotPlant())
# controller = builder.AddSystem(EnergyShapingController(acrobot))
controller = builder.AddSystem(SwingUpAndBalanceController(acrobot))
builder.Connect(acrobot.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), acrobot.get_input_port(0))

# Setup visualization
scene_graph = builder.AddSystem(SceneGraph())
AcrobotGeometry.AddToBuilder(
    builder, acrobot.get_output_port(0), scene_graph
)
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

diagram = builder.Build()
simulator = Simulator(diagram)
context = simulator.get_mutable_context()

if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)

context.SetTime(0.0)
context.SetContinuousState(
    [0, 0, 0.1, 0]
)

simulator.Initialize()
meshcat.StartRecording()
simulator.AdvanceTo(15)
meshcat.StopRecording()
meshcat.PublishRecording()

Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR controller
Using LQR con

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=711f3004-0a9b-43a3-abc8-ecd3b79bb722' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>